In [ ]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA  
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.linear_model import QuantileRegressor  
import datetime as dt
import pandas as pd
import torch
from scipy.stats import binomtest
import scipy.stats as stats
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler 
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,TensorDataset
from collections import Counter
from arch import arch_model
from bayes_opt import BayesianOptimization
from quantile_forest import RandomForestQuantileRegressor
import yfinance as yf
import os  

In [ ]:
# 标普500的 Yahoo 代码是 ^GSPC
sp500 = yf.Ticker("^GSPC")

# 获取历史数据（调整时间范围）
data = sp500.history(
    start="2018-01-01", 
    end="2024-12-31", 
    interval="1d"  # 可选：1d（日线）、1h（小时级）、1m（分钟级）
)
print(data)
print(data.shape)
save_dir = "D:\\Desktop\\学习\\LSTM+QRF（SP500)"
os.makedirs(save_dir, exist_ok=True)  # 自动创建目录

# ========== 保存为CSV ==========
csv_path = os.path.join(save_dir, "sp500_history.csv")
data.to_csv(csv_path, index=True)  # 保留日期索引
print(f"\nCSV文件已保存至: {os.path.abspath(csv_path)}")
dfx = data[['Open', 'High','Low','Volume']]
y1=data[['Close']]
y_log = np.log(y1 / y1.shift(1))
y = y_log.dropna()
import matplotlib.pyplot as plt
import os

# ========== 生成对数收益率图表 ==========
plt.figure(figsize=(14, 7))  # 设置画布尺寸（16:9比例）

# 绘制对数收益率曲线（蓝色实线）
plt.plot(y.index, y['Close'], 
         color='#1f77b4',  # 标准蓝色
         linewidth=1.5, 
         label='Daily Log Returns')

# 添加零基准线（红色虚线）
plt.axhline(0, color='red', linestyle='--', alpha=0.7)

# 图表装饰参数
plt.title('S&P 500 Log Returns (2018-2024)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Log Return', fontsize=12)
plt.grid(True, linestyle=':', alpha=0.6)  # 网格线增强可读性
plt.legend()

# ========== 保存图表文件 ==========
plot_path = os.path.join(save_dir, "D:\\Desktop\\学术\\sp500_log_returns.png")
plt.savefig(
    plot_path,
    dpi=300,                # 印刷级分辨率
    bbox_inches='tight',    # 自动裁剪空白边缘
    facecolor='white'       # 背景色设为纯白
)

# 显示图表（保存后再调用show()）
plt.show()

In [ ]:
model = arch_model(y, vol='Garch', p=1, q=1)
results = model.fit()

# 6. 获取波动率（conditional volatility）
volatility = results.conditional_volatility

# 7. 可视化波动率
plt.figure(figsize=(10, 6))
plt.plot(volatility)
plt.title(f" (GARCH(1, 1))")
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.savefig('D:\\Desktop\\学术\\garch_volatility.png', dpi=300)
plt.show()

# 8. 输出模型的详细估计结果
print(results.summary())
print(volatility)
# 直接转换 volatility 为 DataFrame，保留索引
volatility_col = volatility.to_frame(name='volatility')
dfx1=dfx[1:]
volatility_col = pd.DataFrame(volatility_col, columns=['volatility'])
# 检查转换后的形状和列名
print(volatility_col.shape)  # 应为 (3644, 1)
print(volatility_col.columns)  # 应包含列名 'volatility'
print("dfx1 shape:", dfx1.shape)        # 预期输出 (3645, N)
print("volatility_col shape:", volatility_col.shape)  # 预期输出 (3644, 1)
x = pd.merge(dfx1, volatility_col, left_index=True, right_index=True, how='inner').to_numpy()
y=y.to_numpy()
print(x)
scaler_x = MinMaxScaler(feature_range=(0,1)) 
scaler_y = MinMaxScaler(feature_range=(0,1))  

# 归一化处理（保持与训练集参数一致）
xsc = scaler_x.fit_transform(x)     
ysc = scaler_y.fit_transform(y.reshape(-1,1)) 
def create_dataset(data, labels, time_steps):
    x, y = [], []
    for i in range(time_steps, len(data)):
        x.append(data[i-time_steps:i])
        y.append(labels[i, 0])  # 假设标签在第0列
    return np.array(x), np.array(y)
time_stamp = 30  
x_rolling, y_rolling = create_dataset(xsc, ysc, time_stamp)
n_samples = x_rolling.shape[0]
split_train = int(0.8* n_samples)  
split_test = int(0.9 * n_samples)  
# 分割数据集（新增xpred/ypred）
x_train1, x_test1, x_pred1 = x_rolling[:split_train], x_rolling[split_train:split_test], x_rolling[split_test:]  
y_train1, y_test1, y_pred1 = y_rolling[:split_train], y_rolling[split_train:split_test], y_rolling[split_test:]
# 输出形状
print("训练集形状:", x_train1.shape, y_train1.shape)
print("测试集形状:", x_test1.shape, y_test1.shape)
print("预测集形状:", x_pred1.shape, y_pred1.shape)  

def quantile_loss_factory(q):
    def loss(y_true, y_pred):
        errors = y_true - y_pred
        return tf.reduce_mean(tf.maximum(q * errors, (q - 1) * errors))
    return loss

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from sklearn.ensemble import RandomForestRegressor

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hs=50, num_layers=1, output_size=1, sequence_length=None, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.hs = int(hs)
        self.num_layers =int(num_layers)
        self.input_size = input_size
        self.sequence_length = sequence_length
        self.lstm = nn.LSTM(input_size, hs, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hs, output_size)

    def forward(self, x):
        batch_size = x.size(0)  # 动态计算batch_size
        h0 = torch.zeros(self.num_layers, batch_size, self.hs).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hs).to(x.device)
        
        # LSTM的输出
        output, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        # 取最后一个时间步的隐藏状态
        last_hidden_state = output[:, -1, :]  # 取最后一个时间步的隐藏状态
        out = self.fc(last_hidden_state)
        return out

# 随机森林图计算
def rf_graph(x):   
    G = np.zeros((len(x), len(x))) 
    for i in range(len(x)):      
        G[i, np.where(x == x[i])[0]] = 1
    nodes = Counter(x)
    nodes_num = np.array([nodes[i] for i in x])       
    return G, G / nodes_num.reshape(-1, 1)   

# 获取随机森林的权重（通过叶节点）
def get_rfweight3(rf, x): 
    n = x.shape[0]
    leaf = rf.apply(x)
    ntrees = leaf.shape[1]
    G_unnorm = np.zeros((n, n))
    G_norm = np.zeros((n, n))  
    for i in range(ntrees):     
        tmp1, tmp2 = rf_graph(leaf[:, i])
        G_unnorm += tmp1
        G_norm += tmp2    
    return G_unnorm / ntrees, G_norm / ntrees

# 定义RWN3模型
class RWN3():
    def __init__(self, num_layers, dropout, hs, device='cuda', quantile=0.05):
        self.hs = int(hs)
        self.device = device
        self.quantile = quantile
        self.dropout = float(dropout)
        self.num_layers = int(num_layers)

    def fit(self, x, y, weight,tau,bs,lr,tol, n_iter,d=False,  verbose=True):
        x = torch.FloatTensor(x)
        y = torch.FloatTensor(y)
        weight = torch.FloatTensor(weight)
        lr=float(lr)
        tau=float(tau)
        bs=int(bs)
        tol=float(tol)
        n_iter=int(n_iter)
        q = 1 - self.quantile
        n = x.shape[0]  # 获取样本的总数
        p = x.shape[2]  # 获取特征数，x.shape[2] 代表每个时间步的特征数

        self.fnet = LSTMModel(input_size=p, hs=self.hs, num_layers=self.num_layers, output_size=1, dropout=self.dropout).to(self.device)
        optimizer = torch.optim.Adam([{'params': self.fnet.parameters()}], lr=lr)

        # 量化回归损失函数
        def quantile_loss(y_pred, y_true, q):
            errors = y_true - y_pred.detach()
            max_errors = torch.max(q * errors, (q - 1) * errors)
            return max_errors

        self.loss_count = []
        last_loss = np.inf
        flag = 0
        for i_iter in range(n_iter):
            # 确保采样时不超出数据大小
            csample = np.random.choice(n, bs, replace=False)  # 使用np.random.choice来采样，避免超出索引范围
            tmp_x = x[csample].to(self.device)
            tmp_y = y[csample].to(self.device)
            tmp_w = weight[csample].reshape(bs, -1).T.to(self.device)

            # 确保tmp_x的形状为 (batch_size, sequence_length, input_size)
            tmp_x = tmp_x.view(bs, -1, p)  # Reshape为 (bs, sequence_length, input_size)

            
            

            # 处理 tau != None 时的操作
            if tau is not None:   
                if tmp_w.shape[0] != tmp_w.shape[1]:
                    tmp_w = tmp_w.reshape(bs, -1)

                # 获取 tmp_w 的对角线并调整为与样本维度匹配的形状
                tmp_w_diag = torch.diagonal(tmp_w, dim1=-2, dim2=-1)  # 获取每个样本的对角线元素
                tmp_w_diag = tmp_w_diag.unsqueeze(1).expand_as(tmp_w)  # 扩展为 (837, 50)
                tmp_w = tmp_w - tmp_w_diag 

            tmp_fx = self.fnet(tmp_x)
            tmp_my = torch.tile(tmp_y, (bs, 1))
            tmp_mfx = torch.tile(tmp_fx, (1, bs))

            if tau is None:       
                loss = quantile_loss(tmp_y, tmp_fx.ravel(), q).mean()      
            else:     
                loss1 = quantile_loss(tmp_y, tmp_fx.ravel(), q).mean()
                loss2 = torch.mean(quantile_loss(tmp_y, tmp_fx.ravel(), q) * tmp_w.T) * n / (n - 1)
                loss = tau * loss1 + (1 - tau) * loss2

            self.loss_count.append(loss.data.cpu().tolist())

            if (np.abs(last_loss - loss.data.cpu().numpy()) <= tol) & (i_iter >= 100):
                if verbose:
                    print(f'Algorithm converges for RWN model at iter {i_iter}, loss: {self.loss_count[-1]}')
                flag = 1
                break

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            last_loss = loss.data.cpu().numpy()

        if flag == 0 and verbose:
            print(f'Algorithm may not converge for RWN model, loss: {self.loss_count[-1]}')

    def predict(self, x_new):
        x_new = torch.FloatTensor(x_new).to(self.device)
        return self.fnet(x_new).cpu().data.numpy().ravel()

In [ ]:
x_test1, y_test1 = np.array(x_test1), np.array(y_test1)
x_train1, y_train1 = np.array(x_train1), np.array(y_train1)
x_train_flat1 = x_train1.reshape(x_train1.shape[0], -1)  
x_test_flat1= x_test1.reshape(x_test1.shape[0], -1) 
y_test1=y_test1.reshape(-1,1)

In [ ]:
def count_below(arr, threshold):
    return np.sum(arr < threshold)

def kupiec_test(violations, total_days, significance_level):
    expected_violations = total_days * significance_level
    # 假设 binom_test 是一个已定义的函数，用于二项式检验
    p_value = binomtest(violations, total_days, significance_level).pvalue
    return p_value

def target_function(dropout, hs, n_iter, lr, bs, tol, tau, num_layers,n_estimators,min_samples_split,min_samples_leaf,max_depth):
    quantile = 0.1
    
    # 准备数据（这里你需要确保 x_train1, y_train1, x_test1, y_test1 是全局可访问的或作为参数传入）
    rf = RandomForestQuantileRegressor(n_estimators=int(n_estimators)
                                       ,min_samples_split=int(min_samples_split)
                                       ,min_samples_leaf=int(min_samples_leaf)
                                      ,max_depth=int(max_depth))
                                    
    rf.fit(x_train_flat1, y_train1)
    mrfw, mrfwn = get_rfweight3(rf, x_train_flat1)
    
    # 设置 RWN3 模型的参数
    model_rwn = RWN3(num_layers=int(num_layers), dropout=dropout, hs=int(hs), device="cuda", quantile=quantile)
    model_rwn.fit(x_train1, y_train1, mrfw, tau=tau, d=False, bs=int(bs), n_iter=int(n_iter), lr=float(lr), tol=float(tol), verbose=False)
    
    # 预测结果
    y_pred1 = model_rwn.predict(x_test1)
    y_pred1 = y_pred1.reshape(-1, 1)  # 确保 y_pred1 是列向量
    
    violations = count_below(y_test1, y_pred1)
    total_days = y_test1.shape[0]
    
    # 计算距离
    distance = np.abs((violations / total_days) - quantile)
    p_value= kupiec_test(violations, total_days, quantile)
    
    # 输出检验结果
    if p_value > quantile:
        print("通过了kupiec检验")
    else:
        print("没有通过kupiec检验")
    
    print(f"Iteration p-value: {p_value}")  # 打印每次迭代的 p 值
    
    # 返回 -distance 作为优化目标（因为 BayesianOptimization 默认寻找最大值）
    return -distance

# 更新超参数的搜索空间，删除 quantile
pbounds = {
    'dropout': (0.0, 0.5),
    'hs': (4, 128),
    'n_iter': (500, 2000),
    'lr': (1e-4, 1e-1),
    'bs': (4, 256),
    'tol': (1e-6, 1e-4),
    'tau': (0.0, 1.0),
    'num_layers': (1, 4),
    'n_estimators':(50,200),
    'min_samples_leaf':(1,15),
    'min_samples_split':(2,20),
    'max_depth':(5,30)
}

# 创建贝叶斯优化器对象
optimizer = BayesianOptimization(
    f=target_function,
    pbounds=pbounds,
    random_state=1
)

# 运行贝叶斯优化
optimizer.maximize(init_points=30, n_iter=500)
best_params = optimizer.max['params']
quantile = 0.1  # 保持与优化时一致

# 输出最佳参数和对应的损失
print(optimizer.max)

In [ ]:
total_samples = x_rolling.shape[0]        
train_end = int(total_samples * 0.8)  
test_end = int(total_samples * 0.9)    
val_start = test_end                   
val_end = total_samples - 1           
window_size=30
# 获取贝叶斯优化后的最佳参数
best_params = optimizer.max['params']

# 初始化预测结果容器
predictions = []

# 滚动预测验证集
for current_idx in range(val_start, val_end + 1):
    # 动态训练集: 包含历史所有数据
    x_train_current = x_rolling[:current_idx]      
    y_train_current = y_rolling[:current_idx]      
    x_train_flat = x_train_current.reshape(-1, window_size * 5)
    
    rf = RandomForestQuantileRegressor(
        n_estimators=int(best_params['n_estimators']),
        min_samples_split=int(best_params['min_samples_split']),
        min_samples_leaf=int(best_params['min_samples_leaf']),
        max_depth=int(best_params['max_depth'])
    )
    rf.fit(x_train_flat, y_train_current)
    mrfw, mrfwn = get_rfweight3(rf, x_train_flat)
    
    # --- RWN3模型训练 ---
    model_rwn = RWN3(
        num_layers=int(best_params['num_layers']),
        dropout=best_params['dropout'],
        hs=int(best_params['hs']),
        device="cuda",
        quantile=0.1
    )
    model_rwn.fit(
        x_train_current,
        y_train_current,
        mrfw,
        tau=best_params['tau'],
        d=False,
        bs=int(best_params['bs']),
        n_iter=int(best_params['n_iter']),
        lr=best_params['lr'],
        tol=best_params['tol'],
        verbose=False
    )
    x_next = x_rolling[current_idx].reshape(1,30,5) 
    y_pred = model_rwn.predict(x_next)
    print(y_pred)
    predictions.append(y_pred[0])

# 转换为numpy数组
predictions = np.array(predictions)
print(f"预测完成，验证集预测结果形状: {predictions.shape}")

In [ ]:
predictions = predictions.reshape(-1, 1)  # 确保 y_pred1 是列向量
y_pred1 = y_pred1.reshape(-1, 1) 
violations1 = count_below(y_pred1, predictions)
total_days1 = y_pred1.shape[0]
quantile=0.1
distance1 = np.abs((violations1 / total_days1) - quantile)
p_value1= kupiec_test(violations1, total_days1, quantile)   
    # 输出检验结果
if p_value1 > quantile:
    print("通过了kupiec检验")
else:
    print("没有通过kupiec检验")
print(p_value1)
print(distance1)